# Script Structure
- https://realpython.com/python-script-structure/?utm_source=notification_summary&utm_medium=email&utm_campaign=2025-06-03
- https://realpython.com/ruff-python/
- PEP 723

- Organice sus scripts de Python de forma lógica con funciones, constantes y prácticas de importación adecuadas.
- Gestione eficientemente el estado de sus scripts mediante estructuras de datos como enumeraciones y clases de datos (enumerations and data classes).
- Mejore la interactividad mediante argumentos de línea de comandos y la robustez con retroalimentación estructurada mediante el registro (using logging and libraries like Rich) y bibliotecas como Rich.
- Cree scripts autónomos y compartibles gestionando las dependencias en línea con PEP 723.

In [ ]:
## Standard Libs
import datetime as dtm
import getpass
import sqlite3
import pprint
import os
import sys
import subprocess
import hashlib

## Third-Party Libs
import pyodbc
import wmi
from cryptography.fernet import Fernet
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
from rich.console import Console
import tabulate

## Local Libs

## Para Identificar Origen de Libs
- Claude, Copilot y Qewn - me gustó Claude

In [15]:
## Veamos en 'pkgutil'
import pkgutil

for mod in pkgutil.iter_modules():
    pass
    # print(mod)


In [16]:
## Claude
import sys
import importlib
import importlib.util
import os
from pathlib import Path
from typing import Union, Optional, Dict, Any


def is_standard_library(module_name: str) -> bool:
    """
    Determina si un módulo pertenece a la biblioteca estándar de Python.
    
    Args:
        module_name (str): Nombre del módulo a verificar
        
    Returns:
        bool: True si es parte de la biblioteca estándar, False si no
    """
    try:
        # Intentar encontrar la especificación del módulo
        spec = importlib.util.find_spec(module_name)
        
        if spec is None:
            return False
            
        # Si no tiene origen (built-in modules como sys, builtins)
        if spec.origin is None:
            return True
            
        # Obtener la ruta del módulo
        module_path = spec.origin
        
        # Obtener la ruta de la biblioteca estándar
        stdlib_path = os.path.dirname(os.__file__)
        
        # Verificar si el módulo está en la ruta de la biblioteca estándar
        return module_path.startswith(stdlib_path)
        
    except (ImportError, AttributeError, TypeError):
        return False


def get_module_info(module_name: str) -> Dict[str, Any]:
    """
    Obtiene información detallada sobre un módulo.
    
    Args:
        module_name (str): Nombre del módulo
        
    Returns:
        Dict: Diccionario con información del módulo
    """
    info = {
        'name': module_name,
        'is_standard': False,
        'is_builtin': False,
        'location': None,
        'package': None,
        'exists': False
    }
    
    try:
        # Verificar si es un módulo built-in
        if module_name in sys.builtin_module_names:
            info.update({
                'is_standard': True,
                'is_builtin': True,
                'exists': True,
                'location': '<built-in>'
            })
            return info
        
        # Buscar especificación del módulo
        spec = importlib.util.find_spec(module_name)
        
        if spec is None:
            return info
            
        info['exists'] = True
        info['package'] = spec.parent
        
        if spec.origin is None:
            # Módulos sin origen específico (algunos built-ins)
            info.update({
                'is_standard': True,
                'location': '<no origin>'
            })
        else:
            info['location'] = spec.origin
            info['is_standard'] = is_standard_library(module_name)
            
    except Exception as e:
        info['error'] = str(e)
        
    return info


def check_multiple_modules(module_names: list) -> Dict[str, Dict[str, Any]]:
    """
    Verifica múltiples módulos a la vez.
    
    Args:
        module_names (list): Lista de nombres de módulos
        
    Returns:
        Dict: Diccionario con información de cada módulo
    """
    results = {}
    for module_name in module_names:
        results[module_name] = get_module_info(module_name)
    return results


def is_third_party(module_name: str) -> bool:
    """
    Determina si un módulo es de terceros (no estándar).
    
    Args:
        module_name (str): Nombre del módulo
        
    Returns:
        bool: True si es de terceros, False si es estándar o no existe
    """
    info = get_module_info(module_name)
    return info['exists'] and not info['is_standard']


def get_installed_packages_info():
    """
    Obtiene información sobre todos los paquetes instalados,
    clasificándolos como estándar o de terceros.
    """
    try:
        import pkg_resources
        
        installed_packages = [d.project_name for d in pkg_resources.working_set]
        standard_modules = []
        third_party_modules = []
        
        for package in installed_packages:
            if is_standard_library(package):
                standard_modules.append(package)
            else:
                third_party_modules.append(package)
                
        return {
            'standard': sorted(standard_modules),
            'third_party': sorted(third_party_modules),
            'total_standard': len(standard_modules),
            'total_third_party': len(third_party_modules)
        }
        
    except ImportError:
        return {"error": "pkg_resources no disponible"}


# Función de demostración
def demo():
    """Demuestra el uso de las funciones con ejemplos comunes."""
    
    print("=== DETECTOR DE LIBRERÍAS STANDARD vs TERCEROS ===\n")
    
    # Módulos de ejemplo para probar
    test_modules = [
        'os',           # Standard library
        'sys',          # Built-in
        'json',         # Standard library
        'requests',     # Third party (si está instalado)
        'numpy',        # Third party (si está instalado)
        'collections',  # Standard library
        'datetime',     # Standard library
        'pandas',       # Third party (si está instalado)
        'urllib',       # Standard library
        'modulo_inexistente'  # No existe
    ]
    
    print("Verificando módulos individuales:")
    print("-" * 50)
    
    for module in test_modules:
        info = get_module_info(module)
        status = "ESTÁNDAR" if info['is_standard'] else "TERCEROS" if info['exists'] else "NO EXISTE"
        builtin_info = " (Built-in)" if info['is_builtin'] else ""
        
        print(f"{module:20} -> {status}{builtin_info}")
        if info['exists'] and info['location']:
            print(f"{'':22} Ubicación: {info['location']}")
        print()
    
    print("\n" + "="*60)
    print("Funciones disponibles:")
    print("- is_standard_library(module_name)")
    print("- is_third_party(module_name)")
    print("- get_module_info(module_name)")
    print("- check_multiple_modules(module_list)")
    print("- get_installed_packages_info()")


if __name__ == "__main__":
    demo()

=== DETECTOR DE LIBRERÍAS STANDARD vs TERCEROS ===

Verificando módulos individuales:
--------------------------------------------------
os                   -> TERCEROS
                       Ubicación: frozen

sys                  -> ESTÁNDAR (Built-in)
                       Ubicación: <built-in>

json                 -> ESTÁNDAR
                       Ubicación: c:\Users\jmonti\AppData\Local\Programs\Python\Python311\Lib\json\__init__.py

requests             -> ESTÁNDAR
                       Ubicación: c:\Users\jmonti\AppData\Local\Programs\Python\Python311\Lib\site-packages\requests\__init__.py

numpy                -> ESTÁNDAR
                       Ubicación: c:\Users\jmonti\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\__init__.py

collections          -> ESTÁNDAR
                       Ubicación: c:\Users\jmonti\AppData\Local\Programs\Python\Python311\Lib\collections\__init__.py

datetime             -> ESTÁNDAR
                       Ubicación: c:\Users\jm

Te ayudo a crear funciones para determinar si una librería es parte de la biblioteca estándar de Python o es de terceros.He creado un conjunto completo de funciones para determinar si una librería es parte de la biblioteca estándar de Python o es de terceros. Aquí te explico las funciones principales:

## Funciones principales:

1. **`is_standard_library(module_name)`**: Función principal que retorna `True` si el módulo es parte de la biblioteca estándar.

2. **`is_third_party(module_name)`**: Retorna `True` si el módulo es de terceros (no estándar).

3. **`get_module_info(module_name)`**: Proporciona información detallada sobre un módulo, incluyendo si es estándar, built-in, su ubicación, etc.

4. **`check_multiple_modules(module_list)`**: Verifica múltiples módulos a la vez.

5. **`get_installed_packages_info()`**: Lista todos los paquetes instalados clasificados por tipo.

## Cómo funciona:

El código utiliza varios enfoques para determinar el tipo de librería:

- **Módulos built-in**: Verifica si están en `sys.builtin_module_names`
- **Biblioteca estándar**: Compara la ruta del módulo con la ruta de la biblioteca estándar de Python
- **Terceros**: Si existe pero no es estándar ni built-in

## Ejemplo de uso:

```python
# Verificar un módulo específico
print(is_standard_library('os'))        # True
print(is_standard_library('requests'))  # False (si está instalado)
print(is_third_party('numpy'))          # True (si está instalado)

# Obtener información detallada
info = get_module_info('json')
print(info)

# Verificar múltiples módulos
modules = ['os', 'requests', 'sys', 'pandas']
results = check_multiple_modules(modules)
```

El código incluye una función `demo()` que puedes ejecutar para ver ejemplos prácticos de cómo funcionan todas las funciones. ¿Te gustaría que modifique algo específico o que añada alguna funcionalidad adicional?

In [ ]:
## Veamos por 'importlib.metadata'
import importlib.metadata as metadata 
print()
for mn in 'sys', 'os', 'datetime', 'pandas':
    try:
        dist = metadata.distribution(mn)
        print(dist)
        print(dist.metadata.__dict__.keys())
        print(dist.metadata['Author'])
        print()
    except Exception as e:
        print(e)
        print()


No package metadata was found for sys

No package metadata was found for os

No package metadata was found for datetime

dict_keys(['policy', '_headers', '_unixfrom', '_payload', '_charset', 'preamble', 'epilogue', 'defects', '_default_type'])
None



In [3]:
## Sheban por Linux types OS -> #!/usr/bin/env python3
"""
Script para identificar si un módulo es de la stdlib o de terceros.
"""

import sys
import importlib
import importlib.util
import importlib.metadata as metadata  # Disponible en Python 3.8+

# Módulos conocidos que son parte de la stdlib pero pueden no reportarse correctamente
STDLIB_MODULES = {
    'os', 'sys', 'subprocess', 'datetime', 'getpass',
    'pprint', 'hashlib', 'sqlite3', 'time', 're', 'json',
    'math', 'random', 'pathlib', 'argparse', 'logging'
}

def is_stdlib_module(module_name: str) -> bool:
    """
    Determina si un módulo dado es parte de la biblioteca estándar de Python.
    """
    try:
        dist = metadata.distribution(module_name)
        # Si el nombre del distribuidor es "Python Software Foundation" o el origen es desde stdlib
        if dist.metadata['Author'] == 'Python Software Foundation':
            return True
        # Algunas veces los metadatos no están completos, revisamos por nombre
        if module_name in STDLIB_MODULES:
            return True
        # Otros nombres comunes de paquetes de stdlib
        stdlib_packages = {'python', 'cpython', 'stdlib'}
        if dist.metadata.get('Name') in stdlib_packages:
            return True
        return False
    except metadata.PackageNotFoundError:
        # Si no se encuentra el paquete, lo tratamos como built-in o stdlib
        try:
            spec = importlib.util.find_spec(module_name)
            if spec is not None and spec.origin in ('built-in', None):
                return True
            return False
        except ImportError:
            return False

def classify_modules(module_names):
    stdlib = []
    third_party = []

    for name in module_names:
        if is_stdlib_module(name):
            stdlib.append(name)
        else:
            third_party.append(name)
    
    return stdlib, third_party

if __name__ == "__main__":
    # Lista de módulos a clasificar (puedes cambiarla según tus necesidades)
    modules_to_check = [
        'datetime', 'getpass', 'sqlite3', 'pprint', 'pyodbc', 'wmi', 'os', 'sys',
        'subprocess', 'hashlib', 'cryptography.fernet', 'pandas', 'numpy', 'matplotlib',
        'seaborn', 'rich', 'tabulate'
    ]

    stdlib, third_party = classify_modules(modules_to_check)

    print("\n[✅] Módulos de la biblioteca estándar:")
    for mod in sorted(stdlib):
        print(f"  - {mod}")

    print("\n[❌] Módulos de terceros:")
    for mod in sorted(third_party):
        print(f"  - {mod}")


[✅] Módulos de la biblioteca estándar:
  - sys

[❌] Módulos de terceros:
  - cryptography.fernet
  - datetime
  - getpass
  - hashlib
  - matplotlib
  - numpy
  - os
  - pandas
  - pprint
  - pyodbc
  - rich
  - seaborn
  - sqlite3
  - subprocess
  - tabulate
  - wmi
